In [10]:
import sys, pathlib
sys.path.insert(0, str(pathlib.Path.cwd().parent.parent))  # add repo root
from tsum import tsum
import torch
import json

from ndtools import fun_binary_graph as fbg # ndtools available at github.com/jieunbyun/network-datasets
from ndtools.graphs import build_graph
from pathlib import Path
import networkx as nx   

In [11]:
DATASET = Path("data") 

nodes = json.loads((DATASET / "nodes.json").read_text(encoding="utf-8"))
edges = json.loads((DATASET / "edges.json").read_text(encoding="utf-8"))
probs_dict = json.loads((DATASET / "probs_eq.json").read_text(encoding="utf-8"))

# build base graph
G_base: nx.Graph = build_graph(nodes, edges, probs_dict)

In [ ]:
dests = ['n22', 'n66']

def s_fun(comps_st):
    conn_pop_ratio, sys_st, info = fbg.eval_population_accessibility(comps_st, G_base, dests,
                                                         avg_speed=60.0, # km/h
                                                         target_time_max = 0.25, # hours: it shouldn't take longer than this to reach any destination
                                                         target_pop_max = [0.95, 0.99], # fraction of population that should be reachable at each destination
                                                         length_attr = 'length_km',
                                                         population_attr = 'population',)

    min_comps_st = None
    return conn_pop_ratio, sys_st, min_comps_st

row_names = list(edges.keys()) 
n_state = 2 # binary states of components

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
probs = [[probs_dict[n]['0']['p'], probs_dict[n]['1']['p']] for n in row_names]
probs = torch.tensor(probs, dtype=torch.float32, device=device)


Check system function

In [22]:
comps_st = {eid: 1 for eid in edges.keys()}
conn_pop_ratio, sys_st, details = s_fun(comps_st)
print(f"conn_pop_ratio: {conn_pop_ratio}, sys_st: {sys_st}, details: {details}")

conn_pop_ratio: 1.0, sys_st: 2, details: None


In [ ]:
sys_surv_st = 2
result = tsum.run_rule_extraction_by_mcs(
    sfun=s_fun,
    probs=probs,
    row_names=row_names,
    n_state=n_state,
    output_dir="tsum_res",
    unk_prob_thres=1e-3,
    unk_prob_opt = 'abs',
    sys_surv_st=sys_surv_st
) 